In [1]:
#Inspired by https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3%20-%20Neural%20Networks/recurrent_network.py
# https://github.com/nlintz/TensorFlow-Tutorials/blob/master/07_lstm.ipynb
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# configuration
#                        O * W + b -> 10 labels for each image, O[? 28], W[28 10], B[10]
#                       ^ (O: output 28 vec from 28 vec input)
#                       |
#      +-+  +-+       +--+
#      |1|->|2|-> ... |28| time_step_size = 28
#      +-+  +-+       +--+
#       ^    ^    ...  ^
#       |    |         |
# img1:[28] [28]  ... [28]
# img2:[28] [28]  ... [28]
# img3:[28] [28]  ... [28]
# ...
# img128 or img256 (batch_size or test_size 256)
#      each input size = input_vec_size=lstm_size=28

# configuration variables
input_vec_size = lstm_size = 28
time_step_size = 28

batch_size = 128
test_size = 256

def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.01), name=name)

def model(X, W, B, lstm_size):
    with tf.name_scope('Model'):
        # X, input shape: (batch_size, time_step_size, input_vec_size)
        XT = tf.transpose(X, [1, 0, 2])  # permute time_step_size and batch_size
        # XT shape: (time_step_size, batch_size, input_vec_size)

        XR = tf.reshape(XT, [-1, lstm_size]) # each row has input for each lstm cell (lstm_size=input_vec_size)
        # XR shape: (time_step_size * batch_size, input_vec_size)
        """
        X_split will contain 28 tensors of shape 'batch_size' x 28
        So that, 1st tensor will contain 1st row (28 pixels) of 'batch_size' images
        2nd tensor will contain 2nd row (28 pixels) of 'batch_size' images
        ...
        """
        X_split = tf.split(XR, time_step_size, 0) # split them to time_step_size (28 arrays)
        # Each array shape: (batch_size, input_vec_size)

        # Make lstm with lstm_size (each input vector size)
        """
        https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell
        BasicLSTMCell(num_units, forget_bias=1.0, state_is_tuple=True, activation=None, reuse=None)
        The value of "num_units" is it up to you, too high a value may lead to overfitting
        or a very low value may yield extremely poor results.
        But, the shape of "outputs" depends on "num_units"
        So, define the shape of "weights" accourdingly because "outputs[-1]" and "weights" will be multiplied
        """
        lstm = rnn.BasicLSTMCell(lstm_size, forget_bias=1.0, state_is_tuple=True)

        # Get lstm cell output, time_step_size (28) arrays with lstm_size output: (batch_size, lstm_size)
        outputs, _states = rnn.static_rnn(lstm, X_split, dtype=tf.float32)

        # Linear activation
        # Get the last output
        print("X_split: ", X_split)
        print("outputs: ", outputs)
    return tf.matmul(outputs[-1], W) + B, lstm.state_size # State size to initialize the stat

mnist = input_data.read_data_sets("../../../Datasets/MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
# print(len(trX))
# print(len(trX[0]))
trX = trX.reshape(-1, 28, 28) # (55000, 28, 28)
# print(len(trX))
# print(len(trX[0]))
# print(len(trX[0][0]))
teX = teX.reshape(-1, 28, 28)

Extracting ../../../Datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../../Datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../../Datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../../Datasets/MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
XT = tf.transpose(trX, [1, 0, 2])
print(XT)
XR = tf.reshape(XT, [-1, lstm_size])
print(XR)
X_split = tf.split(XR, time_step_size, 0)
print(X_split)

Tensor("transpose:0", shape=(28, 55000, 28), dtype=float32)
Tensor("Reshape:0", shape=(1540000, 28), dtype=float32)
[<tf.Tensor 'split:0' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:1' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:2' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:3' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:4' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:5' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:6' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:7' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:8' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:9' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:10' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:11' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:12' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:13' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:14' shape=(55000, 28) dtype=float32>, <tf.Tensor 'split:15' shape=(55000, 28) dtype=float32

In [4]:
X = tf.placeholder("float", [None, 28, 28], name='InputData')
Y = tf.placeholder("float", [None, 10], name='LabelData')

# get lstm_size and output 10 labels
W = init_weights([lstm_size, 10], "Weights")
B = init_weights([10], "Bias")
print("W: ", W)
print("B: ", B)

tf.summary.histogram("weights", W)
tf.summary.histogram("biases", B)

py_x, state_size = model(X, W, B, lstm_size)

print("py_x: ", py_x)

with tf.name_scope('Loss'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
tf.summary.scalar("loss", cost) # Create a summary to monitor loss_op tensor

train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

predict_op = tf.argmax(py_x, 1)

prediction = tf.nn.softmax(py_x)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
with tf.name_scope('Accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar("accuracy", accuracy) # Create a summary to monitor accuracy tensor

W:  <tf.Variable 'Weights:0' shape=(28, 10) dtype=float32_ref>
B:  <tf.Variable 'Bias:0' shape=(10,) dtype=float32_ref>
X_split:  [<tf.Tensor 'Model/split:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:1' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:2' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:3' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:4' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:5' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:6' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:7' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:8' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:9' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:10' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:11' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:12' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:13' shape=(?, 28) dtype=float32>, <tf.Tensor 'Model/split:14' shape=(?, 28) dtype=float32>, <tf.Tenso

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [5]:
import shutil, os
if os.path.exists("rnn_mnist_logs_2/"):
    shutil.rmtree("rnn_mnist_logs_2/")

In [6]:
training_epochs = 10

# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()
    
    summary_op = tf.summary.merge_all() # Merge all summaries into a single op
    summary_writer = tf.summary.FileWriter("rnn_mnist_logs_2/", graph=tf.get_default_graph())
    
    saver = tf.train.Saver(max_to_keep=2)

    for epoch in range(training_epochs):
        for start, end in zip(range(0, len(trX), batch_size), range(batch_size, len(trX)+1, batch_size)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
            # Write logs at every iteration
            summary_str = sess.run(summary_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
            summary_writer.add_summary(summary_str, epoch)
            
        # Save checkpoint
        saver.save(sess, "rnn_mnist_logs_2/model-checkpoint", epoch)

        test_indices = np.arange(len(teX))  # Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]

        print(epoch, np.mean(np.argmax(teY[test_indices], axis=1) == sess.run(predict_op, feed_dict={X: teX[test_indices]})))

0 0.64453125
1 0.80078125
2 0.87109375
3 0.92578125
4 0.90234375
5 0.95703125
6 0.96875
7 0.9765625
8 0.97265625
9 0.98046875
